In [9]:
import argparse
import sys

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

from logger import Logger


class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


@torch.no_grad()
def test(model, data, split_idx, evaluator, node_list):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    # Create a mask for the specific nodes
    mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    mask[node_list] = True  # Set the specific nodes to True

    # Ensure split indices are tensors
    train_idx = torch.tensor(split_idx['train'], dtype=torch.long)
    valid_idx = torch.tensor(split_idx['valid'], dtype=torch.long)
    test_idx = torch.tensor(split_idx['test'], dtype=torch.long)

    # Use the mask to filter indices correctly for each split
    train_mask = mask[train_idx]
    valid_mask = mask[valid_idx]
    test_mask = mask[test_idx]

    # Filter the true and predicted labels
    y_true_train = data.y[train_idx][train_mask]
    y_pred_train = y_pred[train_idx][train_mask]

    y_true_valid = data.y[valid_idx][valid_mask]
    y_pred_valid = y_pred[valid_idx][valid_mask]

    y_true_test = data.y[test_idx][test_mask]
    y_pred_test = y_pred[test_idx][test_mask]

    # Debugging outputs
    print(f"Train: {y_true_train.numel()}, {y_pred_train.numel()}")
    print(f"Valid: {y_true_valid.numel()}, {y_pred_valid.numel()}")
    print(f"Test: {y_true_test.numel()}, {y_pred_test.numel()}")

    # Evaluate using the filtered true and predicted labels
    train_acc = evaluator.eval({'y_true': y_true_train, 'y_pred': y_pred_train})['acc'] if y_true_train.numel() > 0 else 0.0
    valid_acc = evaluator.eval({'y_true': y_true_valid, 'y_pred': y_pred_valid})['acc'] if y_true_valid.numel() > 0 else 0.0
    test_acc = evaluator.eval({'y_true': y_true_test, 'y_pred': y_pred_test})['acc'] if y_true_test.numel() > 0 else 0.0

    return train_acc, valid_acc, test_acc



In [10]:
def main():
    # Detect if running in Jupyter and simulate argument parsing
    if "ipykernel" in sys.argv[0]:
        class Args:
            device = 0
            log_steps = 1
            use_sage = False
            num_layers = 3
            hidden_channels = 256
            dropout = 0.5
            lr = 0.01
            epochs = 1
            runs = 1
        args = Args()
    else:
        # Argument parsing for command-line execution
        parser = argparse.ArgumentParser(description='OGBN-Arxiv (GNN)')
        parser.add_argument('--device', type=int, default=0)
        parser.add_argument('--log_steps', type=int, default=1)
        parser.add_argument('--use_sage', action='store_true')
        parser.add_argument('--num_layers', type=int, default=3)
        parser.add_argument('--hidden_channels', type=int, default=256)
        parser.add_argument('--dropout', type=float, default=0.5)
        parser.add_argument('--lr', type=float, default=0.01)
        parser.add_argument('--epochs', type=int, default=500)
        parser.add_argument('--runs', type=int, default=10)
        args = parser.parse_args()
    
    print(args)

    # Your main code (rest of your logic)...
    device = f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)

    node_list = [110223, 146929, 2940, 104544, 62326, 29759, 96890, 47025, 117732, 163206]
    
    dataset = PygNodePropPredDataset(name='ogbn-arxiv', transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    data = data.to(device)

    split_idx = dataset.get_idx_split()

    # Filter the split indices based on your `node_list`
    train_idx = torch.tensor([n for n in split_idx['train'] if n in node_list], device=device)
    valid_idx = torch.tensor([n for n in split_idx['valid'] if n in node_list], device=device)
    test_idx = torch.tensor([n for n in split_idx['test'] if n in node_list], device=device)

    print(train_idx)
    print(valid_idx)
    print(test_idx)

    if args.use_sage:
        model = SAGE(data.num_features, args.hidden_channels,
                     dataset.num_classes, args.num_layers,
                     args.dropout).to(device)
    else:
        model = GCN(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout).to(device)

    evaluator = Evaluator(name='ogbn-arxiv')
    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, {'train': train_idx, 'valid': valid_idx, 'test': test_idx}, evaluator, node_list)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}%, '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()

if __name__ == "__main__":
    main()

<__main__.main.<locals>.Args object at 0x7efc0042ecd0>


/home/kriti-arora/.local/lib/python3.8/site-packages/ogb/nodeproppred/dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(se

tensor([  2940,  47025,  96890, 104544, 110223, 163206])
tensor([])
tensor([ 29759,  62326, 117732, 146929])
Train: 6, 6
Valid: 0, 0
Test: 4, 4
Run: 01, Epoch: 01, Loss: 3.9765, Train: 33.33%, Valid: 0.00%, Test: 25.00%
Run 01:
Highest Train: 33.33
Highest Valid: 0.00
  Final Train: 33.33
   Final Test: 25.00
All runs:
Highest Train: 33.33 ± nan
Highest Valid: 0.00 ± nan
  Final Train: 33.33 ± nan
   Final Test: 25.00 ± nan


/tmp/ipykernel_3971/2671334735.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_idx = torch.tensor(split_idx['train'], dtype=torch.long)
/tmp/ipykernel_3971/2671334735.py:105: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  valid_idx = torch.tensor(split_idx['valid'], dtype=torch.long)
/tmp/ipykernel_3971/2671334735.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_idx = torch.tensor(split_idx['test'], dtype=torch.long)
/home/kriti-arora/kriti/college/sem5/ML/project/ML_Project/logger.py:38: UserWarning: std(): degrees o